In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

In [ ]:
# metadata file (small file)
tm = pd.read_csv('../../data_external/tilikum_metadata.csv')
# data file (large file)
td = pd.read_csv('../../data_external/tilikum_20200501.csv')
td.head()

In [ ]:
td.tail()

In [ ]:
dif = pd.to_datetime(td['end_time']) - pd.to_datetime(td['start_time'])
(dif == dif[0]).all()

In [ ]:
td = td.rename(columns={'start_time':'time'}).drop(columns=['end_time','measure_period', 'id'])

In [ ]:
tm.T.loc[['detector_description', 'flow_detector_id', 'detector_make','detector_name', 'facility_description'], :]

In [ ]:
 (td.flow_detector_id==1904).sum()

In [ ]:
tE = td.loc[td['flow_detector_id']==1903, ['time', 'volume']]
tE.index = pd.DatetimeIndex(pd.to_datetime(tE['time'])).tz_convert('US/Pacific')
tE = tE.drop(columns=['time']).rename(columns={'volume':'Eastbound'})
tW = td.loc[td['flow_detector_id']==1905, ['time', 'volume']]
tW.index = pd.DatetimeIndex(pd.to_datetime(tW['time'])).tz_convert('US/Pacific')
tW = tW.drop(columns=['time']).rename(columns={'volume':'Westbound'})

In [ ]:
tE.plot();

In [ ]:
tE['2018-11-25':'2019-06-01'].plot();

In [ ]:
tE['2018-11-25':'2019-06-01'] = np.nan

In [ ]:
tE[tE > 1000] = np.nan
tW[tW > 1000] = np.nan

In [ ]:
t = pd.merge(tE, tW, on='time')
t.plot(alpha=0.7, style=['-',':']);

In [ ]:
t.resample('W').sum().plot(style=['-',':'], title='Weekly bike counts on Tilikum');

In [ ]:
def weekplot(d, onlyweekend=False, title=None):
  weekend = np.where(d.index.weekday < 5, 'Weekday', 'Weekend')
  by_time = d.groupby([weekend, d.index.time]).mean()
  if onlyweekend:
    if title is None: title = 'Bikes per 15-min during weekends'
    by_time.loc['Weekend'].plot(title=title)
  else:
    if title is None: title = 'Bikes per 15-min during weekdays'
    by_time.loc['Weekday'].plot(title=title)
weekplot(t)

In [ ]:
weekplot(t, onlyweekend=True)

In [ ]:
weekplot(t.loc[:'2020-03-17'], title='Before social distancing')

In [ ]:
weekplot(t.loc['2020-03-17':], title='After social distancing')

In [ ]:
t.loc[:'2020-03-17'].mean() - t.loc['2020-03-17':].mean()

In [ ]:
t.loc[:'2020-03-17'].resample('W').sum().mean() - t.loc['2020-03-17':].resample('W').sum().mean()

In [21]:
import os
import shutil
import urllib
url = "https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD"
f = "../../data_external/Fremont_Bridge_Bicycle_Counter.csv"
if not os.path.isdir('../../data_external/'):
  os.mkdir('../../data_external/')
if not os.path.exists(f):
  with open(f, 'wb') as fo:
    r = urllib.request.urlopen(url)
    shutil.copyfileobj(r, fo)

In [22]:
sd = pd.read_csv(f)
sd.tail()

,Date,"Fremont Bridge Sidewalks, south of N 34th St","Fremont Bridge Sidewalks, south of N 34th St Cyclist East Sidewalk","Fremont Bridge Sidewalks, south of N 34th St Cyclist West Sidewalk"
95635,08/31/2023 07:00:00 PM,224.0,72.0,152.0
95636,08/31/2023 08:00:00 PM,142.0,59.0,83.0
95637,08/31/2023 09:00:00 PM,67.0,35.0,32.0
95638,08/31/2023 10:00:00 PM,43.0,18.0,25.0
95639,08/31/2023 11:00:00 PM,12.0,8.0,4.0


In [ ]:
sd = sd.rename(columns={'Date' : 'time',
'Fremont Bridge East Sidewalk' : 'East',
'Fremont Bridge West Sidewalk' : 'West'})
sd.index = pd.to_datetime(sd.loc[:, 'time'])
sd = sd.drop(columns=['time', 'Fremont Bridge Total'])
sd.head()

In [ ]:
th = t.resample('H').sum()
th.describe() # Portland's Tilikum

In [25]:
sd.describe() # Seattle's Fremont

,"Fremont Bridge Sidewalks, south of N 34th St","Fremont Bridge Sidewalks, south of N 34th St Cyclist East Sidewalk","Fremont Bridge Sidewalks, south of N 34th St Cyclist West Sidewalk"
count,95614.000000,95614.000000,95614.000000
mean,106.483036,47.184847,59.298189
std,133.257293,60.166192,83.020651
min,0.000000,0.000000,0.000000
25%,13.000000,6.000000,7.000000
50%,59.000000,27.000000,31.000000
75%,144.000000,65.000000,76.000000
max,1097.000000,698.000000,850.000000


In [ ]:
sw = sd.resample('W').sum()
tw = t.resample('W').sum()
fig, axs = plt.subplots(1, 2, figsize=(13, 3), sharey=True)
plt.subplots_adjust(wspace=0.05)
sw.plot(ax=axs[0], title='Fremont bridge (Seattle) bikes/week');
tw.plot(ax=axs[1], title='Tilikum bridge (Portland) bikes/week');

In [ ]:
weekplot(sd, title='Fremont (Seattle) on weekdays (Bikes/hr)')
weekplot(th, title='Tilikum (Portland) on weekdays (Bikes/hr)')

In [ ]:
weekplot(sd['2020-03-17':], title='Fremont (Seattle): Weekdays after social distancing'); plt.ylabel('Bikes/hour');
weekplot(th['2020-03-17':], title='Tilikum (Portland): Weekdays after social distancing'); plt.ylabel('Bikes/hour');

In [ ]:
import os
import shutil
import urllib
url = "http://bikeped.trec.pdx.edu/"
f = "../../data_external/Fremont_Bridge_Bicycle_Counter.csv"
if not os.path.isdir('../../data_external/'):
  os.mkdir('../../data_external/')
if not os.path.exists(f):
  with open(f, 'wb') as fo:
    r = urllib.request.urlopen(url)
    shutil.copyfileobj(r, fo)